In [60]:
import sys
import json
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [61]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [62]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [63]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [64]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())

# inputs

In [65]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

road_nodes: 100%|███████████████████████████████| 25/25 [00:01<00:00, 13.81it/s]


volumes

In [66]:
import random
random.seed(42)
num_od = 100
origins = [sm.zones.index[random.randint(0,len(sm.zones))] for i in range(num_od)]
destinations = [sm.zones.index[random.randint(0,len(sm.zones))] for i in range(num_od)]
v1 = [random.randint(0,1000) for i in range(num_od)]
v2 = [random.randint(0,1000) for i in range(num_od)]
arr = np.array([origins,destinations,v1,v2]).T
volumes = pd.DataFrame(arr,columns=['origin','destination','car','pt'])
volumes.index.name='index'

# pathfinder

In [67]:
od_set = set(zip(volumes['origin'], volumes['destination']))


In [68]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster!
)


start publicpathfinder
build_mode_combinations
find_broken_mode_paths


breaking modes: set() : 100%|█████████████████████| 4/4 [00:08<00:00,  2.19s/it]

path_analysis


In [69]:

sm.step_road_pathfinder(method='aon', access_time='time',od_set=od_set,path_analysis=True,)

self.volumes does not exist. od generated with self.zones


In [70]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

path_analysis: 100%|████████████████████████| 248/248 [00:00<00:00, 9539.68it/s]


In [71]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

car_los: 100%|██████████████████████████████████| 28/28 [00:03<00:00,  7.11it/s]
